In [1]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20003, done.
remote: Counting objects: 100% (4815/4815), done.
remote: Compressing objects: 100% (4813/4813), done.
remote: Total 20003 (delta 4), reused 4809 (delta 2), pack-reused 15188 (from 3)
Receiving objects: 100% (20003/20003), 277.05 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (567/567), done.
Updating files: 100% (4917/4917), done.


In [2]:
import os
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in sys.modules:
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif 'kaggle_secrets' in sys.modules:
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name
def load_secret(key_name: str) -> Optional[str]:
    """
    Loads a secret key from the appropriate environment (Colab, Kaggle, or local env vars).

    Args:
        key_name (str): The name of the secret key to load (e.g., "WANDB_API_KEY", "HF_TOKEN").

    Returns:
        Optional[str]: The secret key value if found, otherwise None.
    """
    env = ENV_NAME
    secret_value = None

    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")

    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else: # Local environment
            secret_value = os.getenv(key_name)

        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None

        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value

    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
HF_TOKEN = load_secret("HF_TOKEN")

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/
Attempting to load secret 'HF_TOKEN' from 'colab' environment...
✅ Successfully loaded secret 'HF_TOKEN'.


In [3]:
!uv pip install --upgrade pip
!uv pip install -r FontDiffusion/requirements.txt
!uv pip install gdown
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
print("\n⬇️ Installing PyTorch 1.13 (Required for this model)...")
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117

# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install wandb
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 25ms
Audited 1 package in 0.15ms
Using Python 3.12.12 environment at: /usr
Resolved 116 packages in 71ms
  × Failed to build `tokenizers==0.13.3`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta.build_wheel` failed (exit status: 1)

      [stdout]
      running bdist_wheel
      running build
      running build_py
      copying py_src/tokenizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers
      copying py_src/tokenizers/models/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/models
      copying py_src/tokenizers/decoders/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/decoders
      copying py_src/tokenizers/normalizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/normalizers
      copying py_src/tokenizers/pre_tokenizers/__init__.py ->
      build/lib.linux-x86_64-cpython-312/tokenizers/pre_tokenizers
      co

In [4]:
import gdown
%cd {OUTPUT_PATH}
if not os.path.exists("ckpt"):
  url = "https://drive.google.com/drive/folders/12hfuZ9MQvXqcteNuz7JQ2B_mUcTr-5jZ"
  gdown.download_folder(url, quiet=True, use_cookies=False)

/content


In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

Unzipping /content/my_dataset.zip...
Archive:  /content/my_dataset.zip
  inflating: ./my_dataset/results.json  
  inflating: ./my_dataset/results_checkpoint.json  
  inflating: ./my_dataset/sampling/example_content.jpg  
  inflating: ./my_dataset/sampling/example_style.jpg  
  inflating: ./my_dataset/train/ContentImage/char0.png  
  inflating: ./my_dataset/train/ContentImage/char1.png  
  inflating: ./my_dataset/train/ContentImage/char10.png  
  inflating: ./my_dataset/train/ContentImage/char100.png  
 extracting: ./my_dataset/train/ContentImage/char101.png  
  inflating: ./my_dataset/train/ContentImage/char102.png  
  inflating: ./my_dataset/train/ContentImage/char103.png  
  inflating: ./my_dataset/train/ContentImage/char104.png  
  inflating: ./my_dataset/train/ContentImage/char105.png  
  inflating: ./my_dataset/train/ContentImage/char106.png  
  inflating: ./my_dataset/train/ContentImage/char107.png  
  inflating: ./my_dataset/train/ContentImage/char108.png  
  inflating: ./my_dat

In [6]:
# @title Checking checkpoint files (.pth)
import os
import time

CHECKPOINT_DIR = os.path.join(INPUT_PATH, "ckpt")
print(CHECKPOINT_DIR)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
required_files = ["unet.pth", "content_encoder.pth", "style_encoder.pth"]
while True:
    missing = [f for f in required_files if not os.path.exists(f"{CHECKPOINT_DIR}/{f}")]
    if not missing:
        print("\n✅ All weights found! You can proceed to the next step.")
        break
    else:
        print(f"Waiting for files... Missing: {missing}")
        print("Upload them to the 'ckpt' folder now.")
        time.sleep(10) # Checks every 10 seconds

/content/ckpt

✅ All weights found! You can proceed to the next step.


In [7]:
import pandas as pd
import os

def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.

    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return

    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return

    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return

    all_characters = []
    # Ensure the column values are treated as strings before iterating over them
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)

    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")

# --- Example Usage (demonstration with a dummy file) ---
# As the original file 'Ds_300_ChuNom_TuTao.csv' was not found in the previous execution,
# let's create a dummy file to demonstrate the function's usage.
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(INPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")

# Create a dummy CSV file
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")

convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)

# --- How to use with your actual file ---
# Uncomment the lines below and replace 'your_actual_file.csv' and 'your_output.txt'
# with the correct paths for your use case.
#
# original_csv_file = os.path.join(INPUT_PATH, "Ds_300_ChuNom_TuTao.csv") # Or the full path to your CSV
# original_output_txt = os.path.join(OUTPUT_PATH, "nom_tu_tao.txt") # Or your desired output path
# convert_csv_to_chars_txt(original_csv_file, original_output_txt)



--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /content/dummy_data.csv
Successfully converted '/content/dummy_data.csv' to '/content/dummy_chars.txt', with one character per line.


In [8]:
from datasets import load_dataset
from huggingface_hub import login
login(HF_TOKEN)
# Replace with your Hugging Face username and the repo name you chose

repo_name = "font-diffusion-generated-data"
print(f"Downloading dataset from {repo_name}...")

# This downloads the data to a local cache directory (e.g., /root/.cache/huggingface/datasets)
# and returns a Dataset object.
my_dataset = load_dataset(repo_name, split="train")

print("✅ Dataset loaded.")
print(my_dataset) # You can inspect the dataset object

DatasetNotFoundError: Dataset 'font-diffusion-generated-data' doesn't exist on the Hub or cannot be accessed.

In [ ]:
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --start_line 1 \
    --end_line 50 \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts" \
    --output_dir "my_dataset" \
    --resume_from "my_dataset/results_checkpoint.json" \
    --batch_size 24 \
    --save_interval 5 \
    --channels_last \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --seed 42 \
    --compile \
    --enable_xformers

In [11]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

37

In [13]:
!wandb login
!accelerate launch FontDiffusion/train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_1" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=2000 \
    --ckpt_interval=1000 \
    --gradient_accumulation_steps=1 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler="linear" \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision="no"

wandb: Currently logged in as: dungngocpham171 (dungngocpham171-university-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
pygame 2.6.1 (SDL 2.28.4, Python 3.12.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Load the down block  DownBlock2D
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross attention dim in Down Block 2 layer is 1024
Load the down block  MCADownBlock2D
The style_attention cross attention dim in Down Block 1 layer is 1024
The style_attention cross atten

In [14]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="outputs")
    except Exception as e:
        print(f"❌ An error occurred: {e}")

🔍 Found 1 result folder(s) to zip.
   -> Zipping folder: outputs...
   ✅ Created ZIP: outputs.zip

✅ DONE! Successfully zipped 1 out of 1 folder(s).


In [ ]:
import json
from pathlib import Path
from typing import Dict

# Import Hugging Face libraries
from datasets import load_dataset, Dataset, Features, Value, Image

def generate_metadata_from_results(output_dir: Path, results_filename: str = "results.json") -> Path:
    """
    Generates a metadata.jsonl file assuming the generation script was run
    from the parent directory of `output_dir`.

    This is designed for a common Colab/Kaggle workflow where scripts are run
    from `/content/` and outputs are saved to `/content/my_dataset/`.

    Args:
        output_dir (Path): The root directory of the generated output
                           (e.g., Path("/content/my_dataset")).
        results_filename (str): The name of the JSON file containing generation logs.

    Returns:
        Path: The path to the newly created metadata.jsonl file.
    """
    results_file = output_dir / results_filename
    metadata_file = output_dir / "metadata.jsonl"

    # The directory from which the original script was run (e.g., /content/)
    execution_context_dir = output_dir.parent

    print(f"Reading generation data from: {results_file}")
    if not results_file.is_file():
        raise FileNotFoundError(f"The results file was not found at {results_file}")

    with results_file.open("r", encoding="utf-8") as f:
        results_data = json.load(f)

    print(f"Generating metadata file at: {metadata_file}")
    records_written = 0
    with metadata_file.open("w", encoding="utf-8") as f:
        for gen_info in results_data.get("generations", []):
            try:
                # Path from JSON, e.g., "my_dataset/TargetImage.png/..."
                path_from_json = gen_info["output_path"]

                # --- THIS IS THE KEY LOGIC ---
                # 1. Reconstruct the full, absolute path.
                #    Combines the execution context with the relative path from the file.
                #    e.g., Path("/content") + "my_dataset/..." -> Path("/content/my_dataset/...")
                absolute_path = (execution_context_dir / path_from_json).resolve()

                # 2. Make the path relative to the dataset's root directory.
                #    This makes the final metadata portable.
                #    e.g., Path("/content/my_dataset/...").relative_to(Path("/content/my_dataset"))
                #          -> "TargetImage.png/..."
                relative_path_for_metadata = absolute_path.relative_to(output_dir)
                # ---------------------------

                metadata_entry = {
                    "file_name": str(relative_path_for_metadata),
                    "character": gen_info.get("character", "unknown"),
                    "style": gen_info.get("style", "unknown"),
                    "font": gen_info.get("font", "unknown"),
                }
                f.write(json.dumps(metadata_entry) + "\n")
                records_written += 1
            except (KeyError, ValueError) as e:
                print(f"Skipping a record due to an error: {e}")

    print(f"✅ Metadata generation complete. {records_written} records written.")
    return metadata_file

# Note: The `load_dataset_from_metadata` function you provided in the last
# prompt does not need to be changed. It is already robust and will work
# perfectly with the output of this new generation function.


def load_dataset_from_metadata(metadata_path: Path, base_data_dir: Path) -> Dataset:
    """
    Loads a Hugging Face Dataset using a metadata.jsonl file.

    This function reads the metadata, resolves the relative image paths,
    and loads the images into a structured Dataset object.

    Args:
        metadata_path (Path): Path to the generated metadata.jsonl file.
        base_data_dir (Path): The root directory where the image files are located.
                              This is used to resolve the relative paths in the metadata.

    Returns:
        Dataset: The loaded and structured Hugging Face Dataset.
    """
    print(f"Loading dataset using metadata: {metadata_path}")
    if not metadata_path.is_file():
        raise FileNotFoundError(f"The metadata file was not found at {metadata_path}")

    # 1. Load the JSON data first. This will create a dataset with a 'file_name' column.
    dataset = load_dataset("json", data_files=str(metadata_path), split="train")

    # 2. Define a function to resolve the relative file_name to a full path for loading.
    #    The `datasets.Image()` feature needs a complete path to open the file.
    def resolve_image_path(example: Dict) -> Dict:
        # Use pathlib's `/` operator for clean path joining
        example["image"] = str(base_data_dir / example["file_name"])
        return example

    print("Resolving image paths...")
    dataset = dataset.map(resolve_image_path)

    # 3. Cast the 'image' column (which now contains full paths) to the Image feature type.
    #    This tells the library to actually load the pixels from the paths.
    print("Casting paths to images...")
    dataset = dataset.cast_column("image", Image())

    print("✅ Dataset loaded successfully with image data.")
    return dataset

In [ ]:
from pathlib import Path
from datasets import load_dataset, Dataset, Features, Value, Image # Ensure all imports
import json # Ensure json is imported

# (Assume you have already defined `load_dataset_from_metadata` from the previous prompt)

# --- Step 1: Define your paths based on the Colab environment ---
# This makes the code clean and easy to read.
ROOT_PATH = Path("/content/")
OUTPUT_DIR_NAME = "my_dataset"
FULL_OUTPUT_PATH = ROOT_PATH / OUTPUT_DIR_NAME

# --- Step 2: Run your generation script from the root path ---
# (This is a placeholder for your actual generation command)
#
# !python FontDiffusion/sample_batch.py \
#     --output_dir {OUTPUT_DIR_NAME} \
#     ... other args ...
#
# This will create the directory /content/my_dataset/ and populate it.

# --- Step 3: Generate the metadata file ---
# The function now correctly understands the path structure.
try:
    print(f"Starting metadata generation for directory: {FULL_OUTPUT_PATH}")
    generated_metadata_path = generate_metadata_from_results(
        output_dir=FULL_OUTPUT_PATH
    )
except FileNotFoundError as e:
    print(f"❌ ERROR: {e}")
    print("Please ensure your generation script has run successfully and created a results.json file.")

# --- Step 4: Load the structured dataset using the metadata ---
# This part is unchanged and works as intended.
try:
    my_structured_dataset = load_dataset_from_metadata(
        metadata_path=generated_metadata_path,
        base_data_dir=FULL_OUTPUT_PATH
    )

    # --- Step 5: Verify and use your dataset ---
    print("\n" + "="*50)
    print("  ✅ DATASET LOADED SUCCESSFULLY")
    print("="*50)
    print(my_structured_dataset)

    print("\n--- Example Record ---")
    if len(my_structured_dataset) > 0:
        example = my_structured_dataset[0]
        print(f"Character: {example['character']}")
        print(f"Style: {example['style']}")
        print(f"Font: {example['font']}")
        print("Image object:", example['image'])

        # In Colab/Jupyter, this will render the image directly in the output
        display(example['image'])
    else:
        print("Dataset is empty. Check for errors during metadata generation.")

except (NameError, FileNotFoundError) as e:
    print(f"❌ ERROR: Could not load the dataset. Did the metadata generation fail? Details: {e}")

In [ ]:
# Install the library if you haven't already
from huggingface_hub import HfApi, notebook_login

# 1. Login to Hugging Face
# This will use the token from your Kaggle/Colab secrets
notebook_login()

# 2. Define your local path and the repository ID on the Hub
# This is the directory containing ContentImage/, TargetImage/, etc.
local_output_dir = Path(OUTPUT_PATH) / "my_dataset"
repo_id = "dzungpham/font-diffusion-generated-data" # Choose a name for your repo

# 3. Create the repository and upload the folder
api = HfApi()

print(f"Creating repository '{repo_id}' on the Hub...")
api.create_repo(
    repo_id=repo_id,
    repo_type="dataset", # Can be 'dataset' or 'model'
    private=True,      # Set to False if you want it public
    exist_ok=True      # Don't fail if it already exists
)

print(f"Uploading folder '{local_output_dir}' to '{repo_id}'...")
# This command will recursively upload everything, preserving the structure.
api.upload_folder(
    folder_path=local_output_dir,
    repo_id=repo_id,
    repo_type="dataset"
)

print("✅ Upload complete! Your file tree is now on the Hub.")